In [1]:
import requests
import time
import json

In [2]:
TOKEN = "7748916044:AAH-wllmjaZptxyGyxO92uscdNmtsq3AToU"
URL = f"https://api.telegram.org/bot{TOKEN}/"

In [3]:
user_states = {}

In [4]:
def get_updates(offset=None):
    params = {"timeout": 100, "offset": offset}
    response = requests.get(URL + "getUpdates", params=params)
    return response.json()

In [5]:
def send_message(chat_id, text, reply_markup=None):
    payload = {"chat_id": chat_id, "text": text}
    if reply_markup:
        payload["reply_markup"] = json.dumps(reply_markup)
    requests.get(URL + "sendMessage", params=payload)

In [6]:
def handle_message(chat_id, text, user_id):
    state = user_states.get(user_id, {}).get("state")

    if text == "/start":
        keyboard = {
            "inline_keyboard": [
                [{"text": "✅ Заполнить анкету", "callback_data": "fill_form"}],
                [{"text": "❌ Выйти", "callback_data": "cancel"}]
            ]
        }
        send_message(
            chat_id,
            "Привет, я бот. Я могу сохранить твою анкету.\nЧто хочешь выбрать?",
            keyboard
        )

    elif text == "/help":
        send_message(chat_id,
            "📋 Команды:\n"
            "▶ /start — Начать работу с ботом\n"
            "▶ /help — Показать все команды"
        )
        
    elif text == "/show":
        try:
            response = requests.get(f"http://localhost:5000/get_user/{user_id}")
            if response.ok:
                user_data = response.json()
                name = user_data.get("name", "—")
                age = user_data.get("age", "—")
                send_message(chat_id, f"🧾 Ваша анкета:\n👤 Имя: {name}\n🎂 Возраст: {age}")
            else:
                send_message(chat_id, "⚠ Анкета не найдена. Сначала заполните её через /start.")
        except:
            send_message(chat_id, "❌ Ошибка при подключении к серверу.")

    elif state == "waiting_name":
        user_states[user_id]["name"] = text
        user_states[user_id]["state"] = "waiting_age"
        send_message(chat_id, "Введите ваш возраст:")

    elif state == "waiting_age":
        user_states[user_id]["age"] = text
        data = {
            "user_id": user_id,
            "name": user_states[user_id]["name"],
            "age": user_states[user_id]["age"]
        }
        response = requests.post("http://localhost:5000/submit", json=data)
        if response.ok:
            send_message(chat_id, "✅ Анкета сохранена!")
        else:
            send_message(chat_id, "❌ Ошибка при сохранении анкеты.")
        user_states.pop(user_id)


In [7]:
def handle_callback_query(callback):
    user_id = callback["from"]["id"]
    chat_id = callback["message"]["chat"]["id"]
    data = callback["data"]

    if data == "fill_form":
        user_states[user_id] = {"state": "waiting_name"}
        send_message(chat_id, "Введите ваше имя:")

    elif data == "cancel":
        send_message(chat_id, "Вы вышли из меню. Напишите /start, чтобы начать заново.")

In [8]:
def main():
    offset = None
    print("Бот запущен...")
    while True:
        updates = get_updates(offset)

        if "result" in updates:
            for update in updates["result"]:
                offset = update["update_id"] + 1

                if "message" in update:
                    message = update["message"]
                    chat_id = message["chat"]["id"]
                    text = message.get("text", "")
                    user_id = message["from"]["id"]
                    handle_message(chat_id, text, user_id)

                if "callback_query" in update:
                    handle_callback_query(update["callback_query"])

        time.sleep(1)

In [ ]:
if __name__ == "__main__":
    main()

Бот запущен...
